In [54]:
import os
import time
import math
import sys
import warnings
import numpy as np
import matplotlib.pyplot as plt
from pyquaternion import Quaternion

sys.path.append("../")
sys.path.append("../deepneuroan")
sys.path.append('/home/ltetrel/DeepNeuroAN/deepneuroan/')
sys.path.append('/home/ltetrel/DeepNeuroAN/')
warnings.filterwarnings('ignore')

from deepneuroan.data_generator import DataGenerator
from deepneuroan.models import ChannelwiseConv3D
from deepneuroan.generate_train_data import generate_random_quaternions, generate_random_transformations, quaternion_from_euler

In [55]:
p_outliers=0.05
n_vol=405
n_transfs=100
range_rad=10*np.pi/180
range_mm=1
seed=0

# Q = generate_random_quaternions(rnd, range_rad, p_outliers, method="uniform")
Q, t = generate_random_transformations(n_transfs, n_vol, p_outliers, range_rad, range_mm, seed)

In [56]:
np.random.seed(seed)
rnd = 2*(np.random.rand(n_vol*n_transfs, 6) - 0.5)

n_samples = int(np.ceil(n_vol*n_transfs))
n_outliers = int(np.ceil(p_outliers * n_vol * n_transfs))
rnd_outliers = np.random.rand(n_outliers, 6)
idx_outliers = np.int32((rnd_outliers*n_samples)[..., 0])
Q[idx_outliers,] = quaternion_from_euler(rnd_outliers, np.pi)
t[idx_outliers,] = rnd_outliers[:, :3]*2

In [43]:
Q_reshaped = np.reshape(Q, (n_vol, n_transfs, 4))
t_reshaped = np.reshape(t, (n_vol, n_transfs, 3))

In [ ]:
angles = np.zeros((Q.shape[0]*Q.shape[1], 3))
for ii in range(Q.shape[0]):
    for jj in range(Q.shape[1]):
        idx = ii*Q.shape[1] + jj
        angles[idx, :] = np.array(Quaternion(Q[ii, jj, :]).yaw_pitch_roll[::-1]) * 180 / math.pi

In [ ]:
ang = angles[:,0]
print(np.max(ang))
print(np.min(ang))
print("mean %f" %(np.mean(ang.flatten())))
print("percentile 95%% %f" %np.percentile(ang.flatten(), 95))
plt.hist(ang.flatten(), 1000)
plt.show()

In [ ]:
#angles
angles = 2*np.arccos(Q[:,:,0])
print(np.max(angles*180/math.pi))
print("mean %f" %(np.mean(angles.flatten())*180/math.pi))
print("percentile 95%% %f" %np.percentile(angles.flatten()*180/math.pi, 95))
plt.hist(angles.flatten()*180/math.pi, 1000)
plt.show()

In [ ]:
vec = Q[:,:,2].flatten()
print(np.max(vec))
print("mean %f" %(np.mean(vec)))
print("percentile 95%% %f" %np.percentile(vec, 95))
plt.hist(vec, 1000)
plt.show()

In [62]:
norms = []
for i in range(Q.shape[0]):
    norms += [np.linalg.norm(Q[i,], axis=-1)]
norms

[1.0,
 1.0,
 1.0,
 1.0,
 0.9999999999999999,
 1.0,
 1.0,
 1.0,
 0.9999999999999999,
 0.9999999999999999,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999999999999999,
 1.0,
 0.9999999999999998,
 1.0,
 0.9999999999999999,
 1.0,
 1.0,
 1.0,
 0.9999999999999999,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999999999999999,
 1.0,
 1.0,
 0.9999999999999999,
 0.9999999999999999,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999999999999999,
 1.0,
 1.0,
 0.9999999999999999,
 0.9999999999999999,
 0.9999999999999999,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0000000000000002,
 0.9999999999999999,
 0.9999999999999999,
 1.0,
 1.0,
 1.0,
 0.9999999999999999,
 1.0,
 1.0,
 1.0,
 0.9999999999999999,
 1.0,
 1.0,
 0.9999999999999998,
 1.0,
 0.9999999999999999,
 0.9999999999999999,
 0.9999999999999999,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999999999999999,
 0.9999999999999999,
 1.0,
 0.9999999999999999,
 1.0,
 0.9999999999999999,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999999999999999,
 0.9999999999999999,
 1.0,
 1.0,
 

In [ ]:
qo = q[0,0,:]

In [ ]:
coeff_inverse = 1/(qo[0]**2 + qo[1]**2 + qo[2]**2 + qo[3]**2)

In [ ]:
qinv = coeff_inverse*np.array([qo[0], -qo[1], -qo[2], -qo[3]])

In [ ]:
qinv

In [ ]:
def q_multiply(q0, q1):
    w0, x0, y0, z0 = q0
    w1, x1, y1, z1 = q1
    q = [-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0
          , x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0
          , -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0
          ,  x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0]
    return q

In [ ]:
q_multiply(qo, qinv)[0]